Extract features from EmoDB dataset

In [1]:
import os
import numpy as np
import pandas as pd
import librosa

# DataFrame with Input Samples

Create Pandas DataFrame with filename and label

In [2]:
label2name = {
    "L": "Boredom",
    "A": "Fear",
    "E": "Disgust",
    "F": "Happiness",
    "T": "Sadness",
    "W": "Anger",
    "N": "Neutral"
}

male = ['03', '10', '11', '12', '15']

dir_path = '/datasets/nicolas_facchinetti/EmoDB/data'
data = []
for filename in os.listdir(dir_path):
    filepath = os.path.join(dir_path, filename)
    
    samples, sr = librosa.load(filepath, sr=22050)
    length = len(samples)/sr
    cat = label2name[filename[5]]
    actor = filename[0:2]
    gender = 'm' if actor in male else 'f'
    data.append([filepath, filename, cat, actor, gender, length])
    
df_info = pd.DataFrame(data, columns=['path', 'filename', 'label', 'actor', 'gender', 'length'])
df_info

,path,filename,label,actor,gender,length
0,/datasets/nicolas_facchinetti/EmoDB/data/13a05...,13a05Tc.wav,Sadness,13,f,4.451383
1,/datasets/nicolas_facchinetti/EmoDB/data/14b09...,14b09Ac.wav,Fear,14,f,2.287710
2,/datasets/nicolas_facchinetti/EmoDB/data/11b02...,11b02Na.wav,Neutral,11,m,2.698821
3,/datasets/nicolas_facchinetti/EmoDB/data/09b03...,09b03Fd.wav,Happiness,09,f,3.312517
4,/datasets/nicolas_facchinetti/EmoDB/data/16b10...,16b10Fb.wav,Happiness,16,f,2.583537
...,...,...,...,...,...,...
530,/datasets/nicolas_facchinetti/EmoDB/data/15a04...,15a04Wa.wav,Anger,15,m,1.945397
531,/datasets/nicolas_facchinetti/EmoDB/data/11b03...,11b03Td.wav,Sadness,11,m,5.905034
532,/datasets/nicolas_facchinetti/EmoDB/data/12b02...,12b02Na.wav,Neutral,12,m,2.982086
533,/datasets/nicolas_facchinetti/EmoDB/data/14b01...,14b01Eb.wav,Disgust,14,f,3.291791


Sampes length analysis

In [3]:
df_info["length"].describe()

count    535.000000
mean       2.779635
std        1.028189
min        1.225533
25%        2.026621
50%        2.590204
75%        3.308435
max        8.978277
Name: length, dtype: float64

In [4]:
df_info.label.value_counts()

Anger        127
Boredom       81
Neutral       79
Happiness     71
Fear          69
Sadness       62
Disgust       46
Name: label, dtype: int64

In [5]:
df_info.actor.value_counts()

16    71
14    69
13    61
08    58
15    56
11    55
03    49
09    43
10    38
12    35
Name: actor, dtype: int64

Save metadata

In [6]:
df_info.to_csv("/datasets/nicolas_facchinetti/processed_data/EmoDB/processed_metadata.csv", index=False)